In [91]:
!pip install requests beautifulsoup4 chromadb PyPDF2 sentence-transformers transformers huggingface_hub  llama-cpp-python jinja2 python-decouple openai==1.56.1 httpx==0.27.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.8/389.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.3 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.0
    Uninstalling httpx-0.28.0:
      Successfully uninstalled httpx-0.28.0
  Attempting uninstall: openai
    Found existing installation: openai 1.54.5
    Uninstalling openai-1.54.5:
      Successfully uninstalled openai-1.54.5


# 1. Setup
Preparacion de directorios y decarga de documentos correspondientes.

In [45]:
!mkdir data
!chmod 777 data

mkdir: cannot create directory ‘data’: File exists


In [28]:
!wget https://czechgames.com/files/rules/lost-ruins-of-arnak-rules-es.pdf -O data/lost-ruins-of-arnak-rules-es.pdf

--2024-12-10 14:47:27--  https://czechgames.com/files/rules/lost-ruins-of-arnak-rules-es.pdf
Resolving czechgames.com (czechgames.com)... 185.3.95.246
Connecting to czechgames.com (czechgames.com)|185.3.95.246|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18602936 (18M) [application/pdf]
Saving to: ‘data/lost-ruins-of-arnak-rules-es.pdf’

data/lost-ruins-of- 100%[===================>]  17.74M  5.88MB/s    in 3.0s    

2024-12-10 14:47:34 (5.88 MB/s) - ‘data/lost-ruins-of-arnak-rules-es.pdf’ saved [18602936/18602936]



In [1]:
import chromadb
import os
import requests
import json
import torch

from openai import OpenAI
from decouple import config
from jinja2 import Template
from transformers import pipeline
from google.colab import userdata
from bs4 import BeautifulSoup
from chromadb.config import Settings
from PyPDF2 import PdfReader
from huggingface_hub import hf_hub_download
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [20]:
import warnings
warnings.filterwarnings('ignore')

# 1. Preparación de los datos

## 1.1 Scrapping
Se realiza el scrapping de las páginas donde se encuentran la información requerida del juego.

In [2]:
def save_webpage_text(url, output_file):
    try:
        response = requests.get(url)
        response.raise_for_status()

        soup = BeautifulSoup(response.content, 'html.parser')

        full_text = soup.get_text(separator="\n", strip=True)

        with open(output_file, 'w', encoding='utf-8') as file:
            file.write(full_text)

        print(f"Texto extraído y guardado en {output_file}")
    except Exception as e:
        print(f"Error: {e}")

In [32]:
save_webpage_text(url="https://misutmeeple.com/2021/04/resena-las-ruinas-perdidas-de-arnak/",
                  output_file=os.path.join(os.getcwd(), 'data', 'resena.txt'))

Texto extraído y guardado en /content/data/resena.txt


In [34]:
save_webpage_text(url="https://donmeeple.com/ruinas-perdidas-arnak-juego-mesa/",
                  output_file=os.path.join(os.getcwd(), 'data', 'resena_2.txt'))

Texto extraído y guardado en /content/data/resena_2.txt


In [37]:
save_webpage_text(url="https://mishigeek.com/lost-ruins-of-arnak-resena-en-espanol/",
                  output_file=os.path.join(os.getcwd(), 'data', 'reglas.txt'))

Error: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


## 1.2 Creación de la base de datos vectorial
Creo todas las funciones necesarias para facilitar el preprocesado de los datos.

In [3]:
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
client = chromadb.PersistentClient(path=os.path.join('data', 'vector'))

collection = client.get_or_create_collection(name="arnak")

In [5]:
def read_text_file(file_path: str) -> str:
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

In [6]:
def read_pdf_file(file_path: str) -> str:
    pdf_reader = PdfReader(file_path)
    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text()
    return text

In [7]:
def split_into_chunks(text: str, chunk_size: int = 500, chunk_overlap: int = 100) -> list[str]:
    clean_text = "\n".join(
        line.strip() for line in text.splitlines() if line.strip()
    )

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", "."]
    )

    chunks = text_splitter.split_text(clean_text)

    return chunks

In [8]:
def insert_into_chromadb(collection: str, chunks: list[str], embedding_model) -> None:
    for i, chunk in enumerate(chunks):
        embedding = embedding_model.encode(chunk)
        collection.add(
            documents=[chunk],
            metadatas=[{"chunk_id": i}],
            ids=[f"doc_{i}"]
        )

In [9]:
def process_and_store(file_paths: str) -> None:
    for file_path in file_paths:
        if file_path.endswith('.txt'):
            text = read_text_file(file_path)
        elif file_path.endswith('.pdf'):
            text = read_pdf_file(file_path)
        else:
            raise ValueError(f"Formato no soportado: {file_path}")

        chunks = split_into_chunks(text)
        insert_into_chromadb(collection, chunks, embedding_model)

In [10]:
file_paths: list[str] = [os.path.join(os.getcwd(), 'data', 'resena.txt'),
                         os.path.join(os.getcwd(), 'data', 'resena_2.txt'),
                         os.path.join(os.getcwd(), 'data', 'lost-ruins-of-arnak-rules-es.pdf')]

In [13]:
process_and_store(file_paths)

## 1.2 Funciones de consulta

In [11]:
def get_embedding(query: str, embedding_model) -> list[float]:
    return embedding_model.encode(query)

In [12]:
def query_chromadb(collection: str, query: str, embedding_model, top_k: int=3) -> list[list[float]]:
    embedding = get_embedding(query, embedding_model)
    results = collection.query(
        query_embeddings=[embedding],
        n_results=top_k
    )
    return results

In [13]:
def display_results(results):
    for i, document in enumerate(results['documents'][0]):
        print(f"Resultado {i+1}:")
        print(f"Texto: {document}")
        print(f"Metadata: {results['metadatas'][0][i]}")
        print(f"ID: {results['ids'][0][i]}")
        print("-" * 50)

In [20]:
query = "Reglas del juego Las Ruinas Perdidas de Arnak"
results = query_chromadb(collection, query, embedding_model)

display_results(results)

Resultado 1:
Texto: Desarrollo de la Partida
Una partida de Las Ruinas Perdidas de Arnak se desarrolla a lo largo de cinco rondas. En cada ronda, comenzando por el jugador inicial y continuando en el sentido de las agujas del reloj, los jugadores alternarán turnos.
En cada turno, el jugador activo escogerá una de las acciones disponibles:
Metadata: {'chunk_id': 45}
ID: doc_45
--------------------------------------------------
Resultado 2:
Texto: Así se nos presenta Las Ruinas Perdidas de Arnak, diseñado por el matrimonio formado por Michal «Elwen» Štach y Michaela “Mín” Štachová, siendo este su primera obra. Fue publicado por primera vez en 2020 por Czech Games Edition (CGE) en una versión en inglés. De las ilustraciones se encargan Jiří Kůs, Ondřej Hrdina, Jakub Politzer, František Sedláček y Milan Vavroň.
Metadata: {'chunk_id': 3}
ID: doc_3
--------------------------------------------------
Resultado 3:
Texto: Las Ruinas Perdidas de Arnak es un juego con mecánicas principales de colo

# 2. Preparación del RAG

## 2.1 Funciones generales

In [14]:
def augment_with_retrieved_data(chat_prompt: list, collection, query: str, embedding_model, top_k: int = 3):
    """
    Embebe información relevante de la base vectorial al prompt.
    """
    retrieval_results = query_chromadb(collection, query, embedding_model, top_k)

    relevant_data = " ".join(result for result in retrieval_results["documents"][0])

    chat_prompt.append({
        "role": "system",
        "content": f"Información adicional relevante: {relevant_data}"
    })

    return chat_prompt

In [15]:
mensajes = [
    {
        "role": "system",
        "content": "Sos un chatbot que se encarga de brindar informacion sobre el juego Las ruinas perdidas de Arnak",
    }
]

In [16]:
preguntas = [
    "¿Qué es el objetivo principal del juego Las Ruinas Perdidas de Arnak?",
    "¿Cuáles son las mecánicas principales de Las Ruinas Perdidas de Arnak?",
    "¿Cómo se juega el modo de exploración en Las Ruinas Perdidas de Arnak?",
    "¿Cuántos jugadores pueden participar en Las Ruinas Perdidas de Arnak?",
    "¿Cuál es la duración aproximada de una partida?",
    "¿Qué componentes incluye el juego?",
    "¿Qué tan complejo es aprender a jugar Las Ruinas Perdidas de Arnak?",
    "¿Existen expansiones o contenido adicional para el juego?",
    "¿Qué estrategia se recomienda al comenzar una partida?",
    "¿Dónde se pueden comprar Las Ruinas Perdidas de Arnak?"
]

## 2.2 RAG con OpenAI

In [17]:
context = mensajes.copy()

In [22]:
def generate_text_with_openai(message):
  global context

  api_key = userdata.get('OPENAI_KEY')

  context = augment_with_retrieved_data(context, collection, message, embedding_model)

  context.append({
      "role": "user",
      "content": message
  })

  client = OpenAI(api_key=api_key)

  response = client.chat.completions.create(model="gpt-3.5-turbo",
                                            messages=context)

  context.append(response.choices[0].message)

  print(f'User: {message}')
  print(f'Chat: {response.choices[0].message.content}')
  print('-'*50, '\n\n')

In [23]:
for pregunta in preguntas:
  generate_text_with_openai(pregunta)

User: ¿Qué es el objetivo principal del juego Las Ruinas Perdidas de Arnak?
Chat: El objetivo principal de Las Ruinas Perdidas de Arnak es explorar las ruinas de una mítica isla en busca de artefactos y conocimiento arcano para poder realizar descubrimientos arqueológicos y avanzar en el track de investigación. Los jugadores deben administrar recursos, mejorar sus mazos de cartas y tomar decisiones estratégicas para acumular puntos de victoria y convertirse en el ganador al finalizar la partida.
-------------------------------------------------- 


User: ¿Cuáles son las mecánicas principales de Las Ruinas Perdidas de Arnak?
Chat: Las Ruinas Perdidas de Arnak es un juego que combina varias mecánicas principales que hacen que la experiencia de juego sea rica y estratégica. Algunas de las mecánicas más destacadas del juego incluyen:

1. Colocación de trabajadores: Los jugadores colocan sus exploradores en diferentes ubicaciones de la isla para obtener recursos, descubrir artefactos y acti